# Finetuning a PLM for MT

This code is available in Hugging Face: 

https://huggingface.co/docs/transformers/tasks/translation

In [1]:
# Install necessary libraries

!pip install transformers datasets evaluate sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
# Login to Hugging Face account

from huggingface_hub import notebook_login

notebook_login()

In [3]:
# Load OPUS Books dataset

from datasets import load_dataset

books = load_dataset("opus_books", "ca-en")

Generating train split:   0%|          | 0/4605 [00:00<?, ? examples/s]

Dataset opus_books downloaded and prepared to /root/.cache/huggingface/datasets/opus_books/ca-en/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
# Split the dataset into a train and test set with the train_test_split method:

books = books["train"].train_test_split(test_size=0.2)

In [5]:
# Take a look at an example:

books["train"][0]

{'id': '99',
 'translation': {'ca': 'Tom no era pas el noi model del poblet.',
  'en': 'He was not the Model Boy of the village.'}}

# Preprocess

In [6]:
# Load a T5 tokenizer to process the English-Catalan language pairs

from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [7]:
# Define preprocessing function

source_lang = "en"
target_lang = "ca"
prefix = "translate English to Catalan: "

def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [8]:
# Use Datasets map method to apply the preprocessing function over the entire dataset

tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/3684 [00:00<?, ? examples/s]

Map:   0%|          | 0/921 [00:00<?, ? examples/s]

In [9]:
# Create a batch of examples using DataCollatorForSeq2Seq

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

# Evaluate

In [10]:
# Load the SacreBLEU metric

import evaluate

metric = evaluate.load("sacrebleu")

In [11]:
# Create a function that passes your predictions and labels to compute to calculate the SacreBLEU score

import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

# Train

In [12]:
# Load T5 with TFAutoModelForSeq2SeqLM

from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [17]:
# Define training hyperparameters

training_args = Seq2SeqTrainingArguments(
    output_dir="t5-mt-en-ca-new",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/content/t5-mt-en-ca-new is already a clone of https://huggingface.co/judithrosell/t5-mt-en-ca-new. Make sure you pull the latest changes with `repo.git_pull()`.


In [18]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,3.689296,0.091700,17.768700
2,No log,3.571062,0.124400,17.659100
3,3.906500,3.500792,0.188900,17.605900
4,3.906500,3.463133,0.219700,17.524400
5,3.728800,3.450840,0.218900,17.539600


TrainOutput(global_step=1155, training_loss=3.7986148454410174, metrics={'train_runtime': 237.4798, 'train_samples_per_second': 77.564, 'train_steps_per_second': 4.864, 'total_flos': 370512984539136.0, 'train_loss': 3.7986148454410174, 'epoch': 5.0})

In [19]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/231M [00:00<?, ?B/s]

Upload file runs/May09_21-46-18_e081963c738c/events.out.tfevents.1683668782.e081963c738c.431.4:   0%|         …

To https://huggingface.co/judithrosell/t5-mt-en-ca-new
   d11f79e..5905bd1  main -> main

   d11f79e..5905bd1  main -> main

To https://huggingface.co/judithrosell/t5-mt-en-ca-new
   5905bd1..215fe8e  main -> main

   5905bd1..215fe8e  main -> main



'https://huggingface.co/judithrosell/t5-mt-en-ca-new/commit/5905bd1f6856c4fd6d20c76a70d952845e19e15e'

# Inference

Now that we’ve finetuned a model, we can use it for inference!

In [20]:
text = "translate English to Catalan: Legumes share resources with nitrogen-fixing bacteria."

The simplest way to try out your finetuned model for inference is to use it in a *pipeline()*. Instantiate a pipeline for translation with your model, and pass your text to it:

In [21]:
from transformers import pipeline

translator = pipeline("translation", model="t5-mt-en-ca-new")
translator(text)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/__init__.py:958: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


[{'translation_text': '-Legumes en el agua en el agua del agua en el agua de bacteria en el azot fixat.'}]

In [25]:
ca_en_books = load_dataset("opus_books", "ca-en")
en_fr_books = load_dataset("opus_books", "en-fr")

  0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

Dataset opus_books downloaded and prepared to /root/.cache/huggingface/datasets/opus_books/en-fr/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
print(len(ca_en_books['train']))
print(len(en_fr_books['train']))

4605
127085
